In [1]:
%pip install --quiet groq
%pip install --quiet langchain
%pip install --quiet chroma-hnswlib
%pip install --quiet HuggingFace
%pip install --quiet -U langchain-community
%pip install --quiet python-dotenv
%pip install --quiet pypdf
%pip install --quiet sentence_transformers
%pip install --quiet chromadb
%pip install --quiet langchain_groq


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys
import chromadb
from dotenv import load_dotenv
load_dotenv()
sys.path.append('../..')

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain_groq import ChatGroq
from IPython.display import clear_output

persist_directory = 'database/chroma'

In [3]:
!rm -rf database/chroma # remove old database files if any

In [4]:
loader = PyPDFLoader("Paper.pdf")
pages = loader.load()

In [5]:
full_text = " ".join([page.page_content for page in pages])

# Initialize the RecursiveCharacterTextSplitter
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=0,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)

# Split the text into chunks
chunks = r_splitter.split_text(full_text)


<>:7: SyntaxWarning: invalid escape sequence '\.'
<>:7: SyntaxWarning: invalid escape sequence '\.'
/var/folders/sd/bjm5gr593mb__5xtlb54wqbr0000gn/T/ipykernel_19893/4004144305.py:7: SyntaxWarning: invalid escape sequence '\.'
  separators=["\n\n", "\n", "(?<=\. )", " ", ""]


In [6]:
model_name = "BAAI/bge-large-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    query_instruction="为这个句子生成表示以用于检索相关文章："
)
model.query_instruction = "为这个句子生成表示以用于检索相关文章："


/opt/homebrew/Cellar/jupyterlab/4.2.3/libexec/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [7]:
# Assuming 'chunks' is a list of strings
documents = [Document(page_content=chunk) for chunk in chunks]

vectordb = Chroma.from_documents(
    documents=documents,  # Pass the list of Document objects
    embedding=model,
    persist_directory=persist_directory,
    client=chromadb.Client()
)

In [8]:
# Get Groq API key
groq_api_key = os.environ['GROQ_API_KEY']
model = 'llama3-8b-8192'
# Initialize Groq Langchain chat object and conversation
groq_chat = ChatGroq(
        groq_api_key=groq_api_key, 
        model_name=model
)

conversational_memory_length = 5 # number of previous messages the chatbot will remember during the conversation

memory = ConversationBufferWindowMemory(k=conversational_memory_length, memory_key="chat_history", return_messages=True)

In [9]:
def main():
    clear_output(wait=True)
    while True:
        user_question = input("Ask a question: ")
        
        # If the user has asked a question,
        if user_question:
            context = vectordb.max_marginal_relevance_search(user_question)
            PROMPT_TEMPLATE = """
            Don't display anything non relavent to user_question.
            Answer the question based on the following context: {context}.
            Provide a detailed answer.
            Don’t justify your answers.
            Don’t give information not mentioned in the CONTEXT INFORMATION.
            Do not say "according to the context" or "mentioned in the context" or similar.
            """

            # Load retrieved context and user query in the prompt template
            prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
            formatted_prompt = prompt_template.format(context=context)

            # Construct a chat prompt template using various components
            prompt = ChatPromptTemplate.from_messages(
                [
                    SystemMessage(content=formatted_prompt),
                    MessagesPlaceholder(variable_name="chat_history"),
                    HumanMessagePromptTemplate.from_template("{human_input}")
                ]
            )

            # Create a conversation chain using the LangChain LLM
            conversation = LLMChain(
                llm=groq_chat,
                prompt=prompt,
                verbose=False,
                memory=memory,
            )

            # The chatbot's answer is generated by sending the full prompt to the Groq API.
            response = conversation.predict(human_input=user_question)
            print("Question:", user_question, "\nChatbot:", response)

if __name__ == "__main__":
    main()

/var/folders/sd/bjm5gr593mb__5xtlb54wqbr0000gn/T/ipykernel_19893/2764353886.py:32: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  conversation = LLMChain(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Question: what is cnn? 
Chatbot: A CNN (Convolutional Neural Network) is a type of neural network that is particularly well-suited for analyzing data that has a grid-like or spatial structure, such as images or videos.
Question: what is MFCC? 
Chatbot: There is no mention of MFCC in the provided context.
